In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import statsmodels.api as sm
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from math import sqrt
from sklearn.metrics import mean_squared_error
from scipy import stats


%matplotlib inline

#Reading the file

data = pd.read_csv(r'C:/Users/Dell/Desktop/Combined1107.csv')

#Creating Dummies

def get_dum(x):
    for elem in qs[x].unique():
        qs[str(elem)] = qs[x] == elem
        
qs = data[['uhid','Gender','MODE_OF_DELIVERY','inout_patient_status','Gestation','LOS','NEOFAX_ERRORS','MED_ADMIN_ERRORS','ESPGHAN','NUTR_ADMIN','RDS',
       'JAUNDICE', 'SEPSIS','baby_type','ANTENATA_STEROIDS']]

#Converting to Binary

def conv(x):
    if x == True:
        return 1
    else:
        return 0
    
#Creqting Dummy Variables    
get_dum('inout_patient_status')
#Deleting the old columns after dummy variable creation
qs.drop('inout_patient_status',axis=1,inplace=True)
get_dum('NEOFAX_ERRORS')
qs = qs.rename(columns={"No_err": "NF No_err", "ERROR": "NF ERROR"})
qs.drop('NEOFAX_ERRORS',axis=1,inplace=True)
get_dum('MED_ADMIN_ERRORS')
qs = qs.rename(columns={"No_err": "MED ADMIN No_err", "ERROR": "MED ADMIN ERROR", "Not eligible" : "MED ADMIN Not eligible"})
qs.drop('MED_ADMIN_ERRORS',axis=1,inplace=True)
get_dum('ESPGHAN')
qs = qs.rename(columns={"No_err": "ESPGHAN No_err", "ERROR": "ESPGHAN ERROR","Not eligible":"ESPGHAN NE"})
get_dum('NUTR_ADMIN')
qs = qs.rename(columns={"No_err": "NUTRI ADMIN No_err", "ERROR": "NUTRI ADMIN ERROR","Not eligible":"NUTRI ADMIN NE"})
get_dum('Gender')
get_dum('RDS')
qs = qs.rename(columns={"False": "RDS False", "True": "RDS True"})
get_dum('JAUNDICE')
qs = qs.rename(columns={"False": "Jaundice False", "True": "Jaundice True"})
get_dum('SEPSIS')
qs = qs.rename(columns={"False": "Sepsis False", "True": "Sepsis True"})
get_dum('MODE_OF_DELIVERY')
get_dum('baby_type')
get_dum('ANTENATA_STEROIDS')
qs = qs.rename(columns={"false": "Antenatal ST False", "true": "Antenatal ST True", "unknown":"Antenatal ST Unknown"})
qs.drop('MODE_OF_DELIVERY',axis=1,inplace=True)
qs.drop('baby_type',axis=1,inplace=True)
#qs.drop('nan',axis=1,inplace=True)
qs.drop('RDS',axis=1,inplace=True)
qs.drop('JAUNDICE',axis=1,inplace=True)
qs.drop('SEPSIS',axis=1,inplace=True)
qs.drop('NUTR_ADMIN',axis=1,inplace=True)
qs.drop('ESPGHAN',axis=1,inplace=True)
qs.drop('Gender',axis=1,inplace=True)
qs.drop('ANTENATA_STEROIDS',axis=1,inplace=True)

l = ['In Born', 'Out Born', 'NF No_err', 'NF ERROR',
        'MED ADMIN No_err', 'MED ADMIN ERROR',
       'ESPGHAN ERROR', 'ESPGHAN No_err', 'NUTRI ADMIN ERROR',
       'NUTRI ADMIN No_err','NUTRI ADMIN No_err' ,'Male', 'Female', 'RDS False', 'RDS True',
       'Jaundice False', 'Jaundice True', 'Sepsis False', 'Sepsis True',
       'LSCS', 'NVD', 'Single', 'Multiple', 'False',
       'True']

#Converting the dummy variable columns to 1s and 0s.
for i in l:
    qs[i] = qs[i].apply(conv)
    
#Model is defined
    
def model_OLS(data):
    data.drop('uhid',axis=1,inplace=True)
    y = data['LOS']
    X = data.drop('LOS',axis=1)
    #X.drop('uhid',axis=1,inplace=True)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_trainA, X_testA, y_trainA, y_testA = train_test_split(X_test, y_test, test_size=0.2, random_state=42)    
    NegBi_model = sm.OLS(y,X)
    NegBi_model1 = sm.OLS(y_test,X_test)
    NegBi_results = NegBi_model.fit()
    NegBi_results1 = NegBi_model1.fit()
    
    results_as_html = NegBi_results.summary().tables[1].as_html()
    dt = pd.read_html(results_as_html, header=0, index_col=0)[0]
    pred = NegBi_results.predict(X_test)
    predA = NegBi_results.predict(X_testA)
    rms = sqrt(mean_squared_error(y_test, (pred)))
    rmsA = sqrt(mean_squared_error(y_testA, (predA)))
    #chi = sum(residuals(NegBi_model, type = "pearson")^2)


    return (NegBi_results.summary(),rms,NegBi_results1.summary(),rmsA,stats.chi2_contingency(X))

model_OLS(qs)

#Press Shift + Enter to run this Block

C:\Users\Dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\Dell\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



(<class 'statsmodels.iolib.summary.Summary'>
 """
                             OLS Regression Results                            
 Dep. Variable:                    LOS   R-squared:                       0.433
 Model:                            OLS   Adj. R-squared:                  0.426
 Method:                 Least Squares   F-statistic:                     64.25
 Date:                Mon, 09 Sep 2019   Prob (F-statistic):          7.67e-125
 Time:                        17:46:29   Log-Likelihood:                -4176.3
 No. Observations:                1107   AIC:                             8381.
 Df Residuals:                    1093   BIC:                             8451.
 Df Model:                          13                                         
 Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
 -------------------------------------------------------------

In [33]:
#press Shift + Enter to run this Block
def model_OLS_log(data):
    y = np.log(data['LOS'])
    X = data.drop('LOS',axis=1)
    #X.drop('uhid',axis=1,inplace=True)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_trainA, X_testA, y_trainA, y_testA = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
    NegBi_model = sm.OLS(y,X)
    NegBi_model1 = sm.OLS(y_test,X_test)
    NegBi_results = NegBi_model.fit()
    NegBi_results1 = NegBi_model1.fit()
    
    results_as_html = NegBi_results.summary().tables[1].as_html()
    dt = pd.read_html(results_as_html, header=0, index_col=0)[0]
    pred = NegBi_results.predict(X_test)
    predA = NegBi_results.predict(X_testA)
    rms = sqrt(mean_squared_error(y_test, np.exp(pred)))
    rmsA = sqrt(mean_squared_error(y_testA, np.exp(predA)))
    #chi = sum(residuals(NegBi_model, type = "pearson")^2)


    return (NegBi_results.summary(),rms,NegBi_results1.summary(),rmsA,stats.chi2_contingency(X))


model_OLS_log(qs)

(<class 'statsmodels.iolib.summary.Summary'>
 """
                             OLS Regression Results                            
 Dep. Variable:                    LOS   R-squared:                       0.547
 Model:                            OLS   Adj. R-squared:                  0.542
 Method:                 Least Squares   F-statistic:                     101.6
 Date:                Mon, 09 Sep 2019   Prob (F-statistic):          1.46e-177
 Time:                        17:46:48   Log-Likelihood:                -1256.1
 No. Observations:                1107   AIC:                             2540.
 Df Residuals:                    1093   BIC:                             2610.
 Df Model:                          13                                         
 Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
 -------------------------------------------------------------

In [35]:
#press Shift + Enter to run this Block

def model_OLS_sqrt(data):
    y = np.sqrt(data['LOS'])
    X = data.drop('LOS',axis=1)
    #X.drop('uhid',axis=1,inplace=True)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_trainA, X_testA, y_trainA, y_testA = train_test_split(X_train, y_train, test_size=0.2, random_state=42)    
    NegBi_model = sm.OLS(y,X)
    NegBi_model1 = sm.OLS(y_test,X_test)
    NegBi_results = NegBi_model.fit()
    NegBi_results1 = NegBi_model1.fit()
    
    results_as_html = NegBi_results.summary().tables[1].as_html()
    dt = pd.read_html(results_as_html, header=0, index_col=0)[0]
    pred = NegBi_results.predict(X_test)
    predA = NegBi_results.predict(X_testA)
    rms = sqrt(mean_squared_error(y_test, np.power(pred,2)))
    rmsA = sqrt(mean_squared_error(y_testA, np.power(predA,2)))
    #chi = sum(residuals(NegBi_model, type = "pearson")^2)


    return (NegBi_results.summary(),rms,NegBi_results1.summary(),rmsA,stats.chi2_contingency(X))

model_OLS_sqrt(qs)

(<class 'statsmodels.iolib.summary.Summary'>
 """
                             OLS Regression Results                            
 Dep. Variable:                    LOS   R-squared:                       0.530
 Model:                            OLS   Adj. R-squared:                  0.525
 Method:                 Least Squares   F-statistic:                     94.95
 Date:                Mon, 09 Sep 2019   Prob (F-statistic):          5.22e-169
 Time:                        17:49:15   Log-Likelihood:                -1693.0
 No. Observations:                1107   AIC:                             3414.
 Df Residuals:                    1093   BIC:                             3484.
 Df Model:                          13                                         
 Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
 -------------------------------------------------------------

In [37]:
#press Shift + Enter to run this Block

def model_OLS_box(data):
    y = np.power((data['LOS']),-0.2)
    X = data.drop('LOS',axis=1)
    #X.drop('uhid',axis=1,inplace=True)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_trainA, X_testA, y_trainA, y_testA = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
    NegBi_model = sm.OLS(y,X)
    NegBi_model1 = sm.OLS(y_test,X_test)
    NegBi_results = NegBi_model.fit()
    NegBi_results1 = NegBi_model1.fit()
    
    results_as_html = NegBi_results.summary().tables[1].as_html()
    dt = pd.read_html(results_as_html, header=0, index_col=0)[0]
    pred = NegBi_results.predict(X_test)
    predA = NegBi_results.predict(X_testA)
    rms = sqrt(mean_squared_error(y_test, np.power(pred,-5)))
    rmsA = sqrt(mean_squared_error(y_testA, np.power(predA,-5)))
    #chi = sum(residuals(NegBi_model, type = "pearson")^2)


    return (NegBi_results.summary(),rms,NegBi_results1.summary(),rmsA,stats.chi2_contingency(X))

model_OLS_box(qs)

(<class 'statsmodels.iolib.summary.Summary'>
 """
                             OLS Regression Results                            
 Dep. Variable:                    LOS   R-squared:                       0.533
 Model:                            OLS   Adj. R-squared:                  0.528
 Method:                 Least Squares   F-statistic:                     96.04
 Date:                Mon, 09 Sep 2019   Prob (F-statistic):          1.93e-170
 Time:                        17:51:32   Log-Likelihood:                 866.70
 No. Observations:                1107   AIC:                            -1705.
 Df Residuals:                    1093   BIC:                            -1635.
 Df Model:                          13                                         
 Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
 -------------------------------------------------------------

In [50]:
#press Shift + Enter to run this Block

def model_GLM_NegBI(data):
    y = data['LOS']
    X = data.drop('LOS',axis=1)
    #X.drop('uhid',axis=1,inplace=True)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_trainA, X_testA, y_trainA, y_testA = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
    NegBi_model = sm.GLM(y,X, family=sm.families.NegativeBinomial())
    NegBi_model1 = sm.GLM(y_test,X_test, family=sm.families.NegativeBinomial())
    NegBi_results = NegBi_model.fit()
    NegBi_results1 = NegBi_model1.fit()
    
    results_as_html = NegBi_results.summary().tables[1].as_html()
    dt = pd.read_html(results_as_html, header=0, index_col=0)[0]
    pred = NegBi_results.predict(X_test)
    predA = NegBi_results.predict(X_testA)
    rms = sqrt(mean_squared_error(y_test, (pred)))
    rmsA = sqrt(mean_squared_error(y_testA, (predA)))
    #chi = sum(residuals(NegBi_model, type = "pearson")^2)


    return (NegBi_results.summary(),rms,NegBi_results1.summary(),rmsA,stats.chi2_contingency(X))

model_GLM_NegBI(qs)

(<class 'statsmodels.iolib.summary.Summary'>
 """
                  Generalized Linear Model Regression Results                  
 Dep. Variable:                    LOS   No. Observations:                 1107
 Model:                            GLM   Df Residuals:                     1093
 Model Family:        NegativeBinomial   Df Model:                           13
 Link Function:                    log   Scale:                          1.0000
 Method:                          IRLS   Log-Likelihood:                -3268.6
 Date:                Mon, 09 Sep 2019   Deviance:                       547.51
 Time:                        18:11:28   Pearson chi2:                 1.02e+03
 No. Iterations:                    13   Covariance Type:             nonrobust
                          coef    std err          z      P>|z|      [0.025      0.975]
 --------------------------------------------------------------------------------------
 Gestation             -0.1924      0.014    -13.886  

In [52]:
#press Shift + Enter to run this Block

def model_gamma(data):
    
    y = data['LOS']
    X = data.drop('LOS',axis=1)
    #X.drop('uhid',axis=1,inplace=True)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_trainA, X_testA, y_trainA, y_testA = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
    NegBi_model = sm.GLM(y,X, family=sm.families.Gamma(link=sm.families.links.log))
    NegBi_model1 = sm.GLM(y_test,X_test, family=sm.families.Gamma(link=sm.families.links.log))
    NegBi_results = NegBi_model.fit()
    NegBi_results1 = NegBi_model1.fit()
    
    results_as_html = NegBi_results.summary().tables[1].as_html()
    dt = pd.read_html(results_as_html, header=0, index_col=0)[0]
    pred = NegBi_results.predict(X_test)
    predA = NegBi_results.predict(X_testA)
    rms = sqrt(mean_squared_error(y_test, (pred)))
    rmsA = sqrt(mean_squared_error(y_testA, (predA)))
    #chi = sum(residuals(NegBi_model, type = "pearson")^2)


    return (NegBi_results.summary(),rms,NegBi_results1.summary(),rmsA,stats.chi2_contingency(X))

model_gamma(qs)

(<class 'statsmodels.iolib.summary.Summary'>
 """
                  Generalized Linear Model Regression Results                  
 Dep. Variable:                    LOS   No. Observations:                 1107
 Model:                            GLM   Df Residuals:                     1093
 Model Family:                   Gamma   Df Model:                           13
 Link Function:                    log   Scale:                          1.1189
 Method:                          IRLS   Log-Likelihood:                -3198.8
 Date:                Mon, 09 Sep 2019   Deviance:                       664.17
 Time:                        18:12:17   Pearson chi2:                 1.22e+03
 No. Iterations:                    31   Covariance Type:             nonrobust
                          coef    std err          z      P>|z|      [0.025      0.975]
 --------------------------------------------------------------------------------------
 Gestation             -0.1892      0.014    -13.794  

In [53]:
#press Shift + Enter to run this Block

def model_poi(data):
    
    y = data['LOS']
    X = data.drop('LOS',axis=1)
    #X.drop('uhid',axis=1,inplace=True)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_trainA, X_testA, y_trainA, y_testA = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
    NegBi_model = sm.GLM(y,X, family=sm.families.Poisson(link=sm.families.links.log))
    NegBi_model1 = sm.GLM(y_test,X_test, family=sm.families.Poisson(link=sm.families.links.log))
    NegBi_results = NegBi_model.fit()
    NegBi_results1 = NegBi_model1.fit()
    
    results_as_html = NegBi_results.summary().tables[1].as_html()
    dt = pd.read_html(results_as_html, header=0, index_col=0)[0]
    pred = NegBi_results.predict(X_test)
    predA = NegBi_results.predict(X_testA)
    rms = sqrt(mean_squared_error(y_test, (pred)))
    rmsA = sqrt(mean_squared_error(y_testA, (predA)))
    #chi = sum(residuals(NegBi_model, type = "pearson")^2)


    return (NegBi_results.summary(),rms,NegBi_results1.summary(),rmsA,stats.chi2_contingency(X))

model_poi(qs)

(<class 'statsmodels.iolib.summary.Summary'>
 """
                  Generalized Linear Model Regression Results                  
 Dep. Variable:                    LOS   No. Observations:                 1107
 Model:                            GLM   Df Residuals:                     1093
 Model Family:                 Poisson   Df Model:                           13
 Link Function:                    log   Scale:                          1.0000
 Method:                          IRLS   Log-Likelihood:                -4685.7
 Date:                Mon, 09 Sep 2019   Deviance:                       5549.5
 Time:                        18:32:49   Pearson chi2:                 8.60e+03
 No. Iterations:                     5   Covariance Type:             nonrobust
                          coef    std err          z      P>|z|      [0.025      0.975]
 --------------------------------------------------------------------------------------
 Gestation             -0.2124      0.004    -58.701  